In [1]:
from collections import deque
import itertools

In [2]:
def get_input(fname="input.txt"):
    with open(fname) as f:
        lines = [line.strip() for line in f.readlines()]
        split_idx = lines.index("")
        return [[int(i) for i in lines[1:split_idx]], [int(i) for i in lines[split_idx+2:]]]

In [3]:
test_data = get_input("test.txt")

In [4]:
test_data

[[9, 2, 6, 3, 1], [5, 8, 4, 7, 10]]

In [5]:
def play(player1_cards, player2_cards):
    q1 = deque(player1_cards)
    q2 = deque(player2_cards)
    while len(q1) * len(q2) > 0:
        c1 = q1.popleft()
        c2 = q2.popleft()
        if c1 > c2:
            q1 += [c1, c2]
        else:
            q2 += [c2, c1]
    return list(q1) + list(q2)

In [6]:
test_result = play(*test_data)

In [7]:
sum((i + 1) * c for i, c in enumerate(reversed(test_result)))

306

In [8]:
input_data = get_input("input.txt")

In [9]:
result = play(*input_data)

In [10]:
sum((i + 1) * c for i, c in enumerate(reversed(result)))

32598

In [11]:
def game_signature(player1_cards, player2_cards):
    return (tuple(player1_cards), tuple(player2_cards))

def play_recursive(player1_cards, player2_cards, game=None, cache={}):
    q1 = deque(player1_cards)
    q2 = deque(player2_cards)
    game_sig = game_signature(q1, q2)
    if game_sig in cache:
        return None, cache[game_sig]
    game_cache = {}
    if len(q1) + len(q2) not in cache:
        cache[len(q1) + len(q2)] = {}
    while len(q1) * len(q2) > 0:
        sig = game_signature(q1, q2)
        if sig in game_cache:
            return None, 0 # player 1 wins
        if sig in cache:
            return None, cache[sig]
        game_cache[sig] = True
        c1 = q1.popleft()
        c2 = q2.popleft()
        if c1 <= len(q1) and c2 <= len(q2):
            _, winner = play_recursive(itertools.islice(q1, 0, c1), itertools.islice(q2, 0, c2), cache=cache)
        elif c1 > c2:
            winner = 0
        else:
            winner = 1
        if winner == 0:
            q1 += [c1, c2]
        else:
            q2 += [c2, c1]
    winner = 1 - int(len(q1) > 0)
    cache[game_sig] = winner
    return list(q1) + list(q2), winner

In [12]:
%%time
play_recursive(*test_data, cache={})

CPU times: user 64 µs, sys: 0 ns, total: 64 µs
Wall time: 65.8 µs


([7, 5, 6, 2, 4, 1, 10, 8, 9, 3], 1)

In [13]:
%%time
result = play_recursive(*input_data, cache={})[0]

CPU times: user 1.92 s, sys: 10.1 ms, total: 1.93 s
Wall time: 1.94 s


In [14]:
sum((i + 1) * c for i, c in enumerate(reversed(result)))

35836